In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../input/train.csv')
print(data.columns)
print(data.head())
data.dropna(axis=0, subset=['revenue'], inplace=True)
y = data.revenue
X = data.drop(['revenue'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.25)

print("X describe")
print(pd.DataFrame(data=X).describe())
print("y describe")
print(pd.DataFrame(data=y).describe())

print(X.columns)
features = X.columns

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')
   id    ...      revenue
0   1    ...     12314651
1   2    ...     95149435
2   3    ...     13092000
3   4    ...     16000000
4   5    ...      3923970

[5 rows x 23 columns]
X describe
                id        budget   popularity      runtime
count  3000.000000  3.000000e+03  3000.000000  2998.000000
mean   1500.500000  2.253133e+07     8.463274   107.856571
std     866.169729  3.702609e+07    12.104000    22.086434
min       1.000000  0.000000e+00     0.000001     0.000000
25%     750.750000  0.000000e+00     4.018053    94.000000
50%    1500.500000  8.000000e+06     7.374861   104.000000
75%    2250.250000  2

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [4]:
train_predictions = my_model.predict(train_X)
test_predictions = my_model.predict(test_X)

print(pd.DataFrame(data=train_predictions).describe())
print(pd.DataFrame(data=test_predictions).describe())
print(pd.DataFrame(data=train_y).describe())
print(pd.DataFrame(data=test_y).describe())

from sklearn.metrics import mean_squared_log_error
print("training Root-Mean-Squared-Logarithmic-Error (RMSLE) : " + str(np.sqrt(mean_squared_log_error(train_predictions, train_y))))
print("testing  Root-Mean-Squared-Logarithmic-Error (RMSLE) : " + str(np.sqrt(mean_squared_log_error(test_predictions, test_y))))

                  0
count  2.250000e+03
mean   6.779294e+07
std    1.212381e+08
min    2.506152e+06
25%    1.109498e+07
50%    2.912298e+07
75%    7.453095e+07
max    1.394839e+09
                  0
count  7.500000e+02
mean   6.808318e+07
std    1.170036e+08
min    2.904431e+06
25%    1.103132e+07
50%    3.289972e+07
75%    7.456963e+07
max    1.101979e+09
                  0
count  2.250000e+03
mean   6.782486e+07
std    1.414354e+08
min    1.000000e+00
25%    2.390862e+06
50%    1.643270e+07
75%    6.836455e+07
max    1.519558e+09
                  0
count  7.500000e+02
mean   6.342883e+07
std    1.251258e+08
min    1.000000e+00
25%    2.320768e+06
50%    1.743106e+07
75%    7.215237e+07
max    9.201000e+08
training Root-Mean-Squared-Logarithmic-Error (RMSLE) : 2.711439862145175
testing  Root-Mean-Squared-Logarithmic-Error (RMSLE) : 2.7841707132685443


# Make Prediction

In [6]:
# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)
print(test_data.columns)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = test_data[features]

# make predictions which we will submit. 
test_preds = my_model.predict(test_X.values)

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.

output = pd.DataFrame({'id': test_data.id,
                       'revenue': test_preds})
output.to_csv('submission.csv', index=False)

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew'],
      dtype='object')
['.ipynb_checkpoints', 'submission.csv', '__notebook_source__.ipynb']
